In [223]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [229]:
df = pd.read_csv('Iris.csv')
df.drop('Id',axis=1,inplace=True)
df['Species'] = df['Species'].apply(lambda x: 0 if x == 'Iris-virginica' else (1 if x == 'Iris-setosa' else 2))
df.head(150)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0
146,6.3,2.5,5.0,1.9,0
147,6.5,3.0,5.2,2.0,0
148,6.2,3.4,5.4,2.3,0


In [230]:
X = df.drop('Species',axis=1)
y = df['Species']

In [231]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size = 0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(120, 4) (30, 4) (120,) (30,)


In [232]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

model = LogisticRegression(multi_class='multinomial')
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [233]:
accuracy_score(y_test,y_pred)

0.9666666666666667

In [234]:
confusion_matrix(y_test,y_pred)

array([[12,  0,  0],
       [ 0, 12,  0],
       [ 1,  0,  5]])

In [235]:
class SoftmaxRegressor:

  def __init__(self, learning_rate, epochs):
    self.y_pred = None
    self.coef_ = None
    self.intercept_ = None
    self.W = None
    self.epochs = epochs
    self.learning_rate = learning_rate

  def softmax(self, X):
    exp_x = np.exp(X - np.max(X, axis=1, keepdims=True))
    return exp_x / exp_x.sum(axis=1, keepdims=True)

  def fit(self, X_train, y_train):
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_train = np.insert(X_train, 0, 1, axis=1)
    num_classes = len(np.unique(y_train))
    self.W = np.zeros((X_train.shape[1], num_classes))

    for i in range(self.epochs):
      scores = X_train.dot(self.W)
      probs = self.softmax(scores)
      error = self.one_hot_encode(y_train, num_classes) - probs
      gradient = (-1 / len(X_train)) * X_train.T.dot(error)
      self.W -= self.learning_rate * gradient

    self.coef_ = self.W[1:, :]
    self.intercept_ = self.W[0, :]

  def one_hot_encode(self, y, num_classes):
    one_hot = np.zeros((len(y), num_classes))
    for i in range(len(y)):
      one_hot[i, y[i]] = 1
    return one_hot

  def predict(self, X_test):
    X_test = np.array(X_test)
    X_test = np.insert(X_test, 0, 1, axis=1)
    scores = X_test.dot(self.W)
    probabilities = self.softmax(scores)
    predictions = np.argmax(probabilities, axis=1)
    return predictions

  def predict_proba(self, X_test):
    X_test = np.array(X_test)
    X_test = np.insert(X_test, 0, 1, axis=1)
    scores = X_test.dot(self.W)
    probabilities = self.softmax(scores)
    return probabilities

  def accuracy(self, y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    correct_predictions = np.sum(y_true == y_pred)
    total_predictions = len(y_true)
    accuracy = correct_predictions / total_predictions
    return accuracy

  def confusion_matrix(self, y_true, y_pred, num_classes):
    cm = np.zeros((num_classes, num_classes), dtype=int)
    for i in range(num_classes):
      for j in range(num_classes):
        cm[i, j] = np.sum((y_true == i) & (y_pred == j))
    
    return cm.T

model = SoftmaxRegressor(learning_rate = 0.5,epochs = 5000)
model.fit(X_train,y_train)
y_pred1 = model.predict(X_test)
model.accuracy(y_pred1,y_test)
model.confusion_matrix(y_pred1,y_test,3)

array([[12,  0,  0],
       [ 0, 12,  0],
       [ 1,  0,  5]])